# Install Required Libraries

In [1]:
! pip install transformers nltk torch


# Import Libraries

In [2]:
import nltk
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from nltk.tokenize import sent_tokenize

nltk.download('punkt')


c:\Users\musta\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\musta\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Load the Pre-trained Model and Tokenizer

In [8]:
from transformers import BartTokenizer, BartForConditionalGeneration

MODEL_NAME = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(MODEL_NAME)
model = BartForConditionalGeneration.from_pretrained(MODEL_NAME)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_l

# Define Preprocessing and Summarization Functions

In [15]:
def preprocess_article(article):
    return sent_tokenize(article)

def extract_summary(article):
    try:
        sentences = preprocess_article(article)
        inputs = tokenizer(
            " ".join(sentences),  # Join sentences as model expects one long string
            padding=True,
            truncation=True,
            max_length=1024,
            return_tensors="pt"
        )
        inputs = {key: val.to(device) for key, val in inputs.items()}
        
        model.eval()
        with torch.no_grad():
            summary_ids = model.generate(
                inputs['input_ids'], 
                max_length=150,  # You can adjust the length
                num_beams=4,     # Optional parameter to enhance quality
                early_stopping=True
            )
        
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        return summary
    except Exception as e:
        return f"Error during summarization: {e}"

# Input Text Example

In [17]:
sample_text = """
At three months old, even future kings of the savannah like to play.

In Senegal, a video of three cubs that would have hardly been imaginable a few years ago, has given hope that the revered West African lion, which has been decimated over time, will be revived.

The US-based wild cat conservation organisation Panthera has released never-before-seen images of a lioness and three cubs spotted in February by a remote cameras in the Niokolo-Koba National Park in southeastern Senegal.

In what Panthera describes as a "thrilling sign of recovery for the critically endangered West African lion", video footage shows the big cat eating an animal carcass while its three-month-old offspring try to imitate it.

They brandish their claws and fangs, testing them out on scraps of wood or their mother's hindquarters.

"This documentation of new lion life... indicates the remarkable recovery of a population on the brink of extinction", the organisation said in a statement.

Panthera has since 2011 been working with Senegal's Department of National Parks on conservation efforts in the Niokolo-Koba park.

In that time, the number of lions in the park has risen from 10 or 15 to around 30, the organisation said.

The West African lion is characterised by a narrow mane and bald appearance, and is genetically distinct from the African or Asian subspecies.

There are between 120 and 375 of them left, according to Panthera.

Revered to the point of being cited in the Senegalese national anthem, West African lions have been decimated by poaching and the gradual loss of their habitat.

Their historic range has shrunk by 99 percent, Panthera said, citing the International Union for Conservation of Nature.

Florence -- the lioness caught on camera, who is thought to be nine or 10 years old -- has contributed to the population's recovery.
"""

summary = extract_summary(sample_text)
print("Extractive Summary:\n", summary)


Extractive Summary:
 Video shows three-month-old West African lion cubs playing in Senegal park. Lioness and cubs spotted in February by remote cameras in Niokolo-Koba National Park. Conservation group Panthera calls it a "thrilling sign of recovery for the critically endangered" species.


# GUI

In [20]:
import tkinter as tk
from tkinter import scrolledtext, messagebox

def run_gui():
    def summarize():
        input_text = input_text_box.get("1.0", tk.END).strip()
        if not input_text:
            messagebox.showwarning("Input Error", "Please enter some text to summarize!")
            return
        summary = extract_summary(input_text)
        output_text_box.delete("1.0", tk.END)
        output_text_box.insert(tk.END, summary)

    root = tk.Tk()
    root.title("Extractive Summarization")
    root.geometry("800x600")

    tk.Label(root, text="Input Text:", font=("Arial", 12)).pack(anchor="w", padx=10, pady=5)
    input_text_box = scrolledtext.ScrolledText(root, wrap=tk.WORD, font=("Arial", 12), height=15)
    input_text_box.pack(fill=tk.BOTH, padx=10, pady=5, expand=True)

    summarize_button = tk.Button(root, text="Summarize", font=("Arial", 14), bg="blue", fg="white", command=summarize)
    summarize_button.pack(pady=10)

    tk.Label(root, text="Extractive Summary:", font=("Arial", 12)).pack(anchor="w", padx=10, pady=5)
    output_text_box = scrolledtext.ScrolledText(root, wrap=tk.WORD, font=("Arial", 12), height=10)
    output_text_box.pack(fill=tk.BOTH, padx=10, pady=5, expand=True)

    root.mainloop()

run_gui()


In [19]:
def summarization_pipeline():
    from IPython.display import display
    input_text = input("Enter text for summarization: ").strip()
    if input_text:
        summary = extract_summary(input_text)
        print("\nExtractive Summary:\n", summary)
    else:
        print("Please enter valid text!")

summarization_pipeline()



Extractive Summary:
 Video shows three-month-old West African lion cubs playing in Senegal park. Lioness and cubs spotted in February by remote cameras in Niokolo-Koba National Park. Conservation group Panthera calls it a "thrilling sign of recovery for the critically endangered" species.
